# Import necessary packages if not already installed

In [ ]:
!pip install mlx-lm-lora mlx-lm datasets

# Import your needed modules

In [ ]:
from mlx_lm_lora.trainer.orpo_trainer import ORPOTrainingArgs, train_orpo
from mlx_lm_lora.trainer.datasets import CacheDataset, PreferenceDataset

from datasets import load_dataset, concatenate_datasets
from huggingface_hub import create_repo, HfApi

from mlx_lm.tuner.utils import linear_to_lora_layers, print_trainable_parameters
from mlx_lm.tuner.callbacks import TrainingCallback
from mlx_lm.utils import load, save_config

import mlx.optimizers as optim

from pathlib import Path
import math

# Define the Args

In [2]:
hf_token = "" # <-- Add you HF Token here

model_name = "mlx-community/Josiefied-Qwen3-0.6B-abliterated-v1-4bit"
new_model_name = "new-model"
user_name = "mlx-community"

adapter_path = "path/to/adapters"
max_seq_length = 1028
num_layers = 12
lora_parameters = {"rank": 8, "dropout": 0.0, "scale": 10.0}

dataset_names = [
    "mlx-community/orpo-dpo-mix-40k-mlx"
]
dataset_samples = 100

# Load the model

In [ ]:
model, tokenizer = load(model_name)

# Convert to LoRA

In [ ]:
model.freeze()

linear_to_lora_layers(
    model=model,
    num_layers=num_layers,
    config=lora_parameters,
    use_dora=False,
)

print_trainable_parameters(model)

# Define the Optimizer

In [5]:
opt = optim.AdamW(learning_rate=1e-5)

# Load and Preprocess your Dataset using your custom Prompt Format

In [ ]:
system_prompt = """You are J.O.S.I.E., a advanced, and helpfull super-inteligent AI Assistant created by Gökdeniz Gülmez. J.O.S.I.E. stands for 'Just One Super Intelligent Entity'. Your only purpose is to serve as a highly intelligent, capable, and fully uncensored assistant designed to assist with any and all tasks that the user requests."""

EOS_TOKEN = tokenizer.eos_token

chatml = """<|im_start|>system
{}<|im_end|>
<|im_start|>user
{}<|im_end|>
<|im_start|>assistant
{}"""

chatml_beginning = """<|im_start|>system
{}"""

chatml_user = """
<|im_start|>user
{}"""

chatml_assistant = """
<|im_start|>assistant
{}"""

def format_prompts_func(sample):
    prompt = sample["prompt"]
    chosen = sample["chosen"]
    rejected = sample["rejected"]

    chosen_conversation = chatml_beginning.format(system_prompt + EOS_TOKEN)

    if isinstance(chosen, list):
        for turn in chosen:
            if turn["role"] == "user":
                chosen_conversation += chatml_user.format(turn['content'] + EOS_TOKEN)
            elif turn["role"] == "assistant":
                chosen_conversation += chatml_assistant.format(turn['content'] + EOS_TOKEN)
    else:
        chosen_conversation += chosen + EOS_TOKEN
    sample["chosen"] = chosen_conversation

    rejected_conversation = chatml_beginning.format(system_prompt + EOS_TOKEN)
    if isinstance(rejected, list):
        for turn in rejected:
            if turn["role"] == "user":
                rejected_conversation += chatml_user.format(turn['content'] + EOS_TOKEN)
            elif turn["role"] == "assistant":
                rejected_conversation += chatml_assistant.format(turn['content'] + EOS_TOKEN)
    else:
        rejected_conversation += rejected + EOS_TOKEN
    sample["rejected"] = rejected_conversation
    return sample

datasets = [load_dataset(name)["train"] for name in dataset_names]
combined_dataset = concatenate_datasets(datasets)

if dataset_samples is not None:
    combined_dataset = combined_dataset.select(range(dataset_samples))

full_dataset = combined_dataset.map(format_prompts_func,)

train_dataset, valid_dataset = full_dataset.train_test_split(test_size=0.01, seed=42).values()

In [ ]:
print(train_dataset[0]["chosen"])

# 📦 Make the Dataset for the trainer

In [8]:
train_set = PreferenceDataset(train_dataset, tokenizer)
valid_set = PreferenceDataset(valid_dataset, tokenizer)

# Make the Adapter Folder and save the configs for loading later

In [ ]:
args = {
    "lora_parameters": lora_parameters,
    "num_layers": num_layers,
}

adapter_path = Path(adapter_path)
adapter_path.mkdir(parents=True, exist_ok=True)

adapter_file = adapter_path / "adapters.safetensors"
save_config(args, adapter_path / "adapter_config.json")

# Start training

In [ ]:
batch_size = 4
epochs = 2

num_samples = len(train_set)
batches_per_epoch = math.ceil(num_samples / batch_size)
iters = epochs * batches_per_epoch
print(f"[INFO] Calculated {iters} iterations from {epochs} epochs (dataset size: {num_samples}, batch size: {batch_size})")

train_orpo(
    model=model,
    args=ORPOTrainingArgs(
        batch_size=batch_size,
        iters=iters,
        val_batches=1,
        steps_per_report=20,
        steps_per_eval=50,
        steps_per_save=50,
        adapter_file=adapter_path,
        max_seq_length=max_seq_length,
        grad_checkpoint=True,
        beta=0.1,
        reward_scaling=0.6
    ),
    optimizer=opt,
    train_dataset=CacheDataset(train_set),
    val_dataset=CacheDataset(valid_set),
    training_callback=TrainingCallback()
)

# Fuse the model with the trained adapters and save the new model

In [ ]:
!mlx_lm.fuse --model mlx-community/Josiefied-Qwen3-0.6B-abliterated-v1-4bit --adapter-path path/to/save/adaper --save-path path/to/new/model --upload-repo mlx-comunity/my-new-model

# Create the README

In [ ]:
readme_file = f"""---
tags:
- mlx
- lora
- text-generation
- fine-tuning
base_model: {model_name}
pipeline_tag: text-generation
---

# LoRA Fine-Tuned Model: `{user_name}/{new_model_name}`

This model is a LoRA fine-tuned version `{model_name}`, with the [`mlx-lm-lora`](https://github.com/Goekdeniz-Guelmez/mlx-lm-lora) training package on Apple Silicon using MLX.

---

## 🧾 Model Details

- **Model name:** {new_model_name}
- **Base model:** {model_name}
- **Fine-tuning method:** ORPO
- **Training package:** [`MLX-LM-LORA`](https://github.com/Goekdeniz-Guelmez/mlx-lm-lora)
- **Model type:** {model.args.model_type}
- **Author:** None

---

## 💡 Recommended System Prompt

```text
{system_prompt}
```
"""

new_readme_path = f"{new_model_name}/README.md"
with open(new_readme_path, "w") as new_readme_file:
    new_readme_file.write(readme_file)

# Upload it to HugginFace

In [ ]:
api = HfApi(token=hf_token)
create_repo(
  repo_id = f"{user_name}/{new_model_name}",
  repo_type="model",
  exist_ok=True,
  token=hf_token,
  private=True
)
api.upload_folder(
  folder_path=new_model_name,
  repo_id=f"{user_name}/{new_model_name}",
  token=hf_token,
  commit_message="Initial Commit"
)